In [1]:
!pip install -q openai py_expression_eval google-api-python-client

In [2]:
from openai import OpenAI
from googleapiclient.discovery import build
from py_expression_eval import Parser
import re, time, os

from kaggle_secrets import UserSecretsClient

In [3]:
# Follow the SO guide to setup your own custom search engine

user_secrets = UserSecretsClient()

cse_key =user_secrets.get_secret("google_cse")
cse_id =user_secrets.get_secret("cse_id")


In [4]:
class CFG:
    model = "gpt-4-turbo"

# Functions

In [5]:
# wrapper for the CSE 

def search(search_term):
    search_result = ""
    service = build("customsearch", "v1", developerKey= cse_key )
    res = service.cse().list(q=search_term, cx = cse_id, num = 10).execute()
    for result in res['items']:
        search_result = search_result + result['snippet']
    return search_result


# Base model

In [6]:
# setup OpenAI API connection

api_key=user_secrets.get_secret("openaivision")
os.environ['OPENAI_API_KEY']= api_key


client = OpenAI(api_key= api_key)

In [7]:
muhprompt = "Where does Mustafa Suleyman work?"

In [8]:
system_prompt1 = "You are a helpful assistant, giving factual information."

In [9]:

messages = [
        { "role": "system", "content": system_prompt1 },
        { "role": "user", "content": muhprompt },
    ]

completion = client.chat.completions.create(
  model = CFG.model,
  messages = messages,
  temperature = 0,
  max_tokens = 1000,
  top_p = 1
)


print(completion.choices[0].message.content)

As of my last update, Mustafa Suleyman is one of the co-founders of DeepMind Technologies, an artificial intelligence company acquired by Google in 2014. He left DeepMind in 2019 and later joined Google to work on AI policy. However, his current position as of 2023 might have changed, so I recommend checking the latest sources for the most current information.


# Agent

In [10]:
# update the system prompt
system_prompt2 = """
Answer the following questions and obey the following commands as best you can.

You have access to the following tools:

Search: Search: useful for when you need to answer questions about current events. You should ask targeted questions.
Response To Human: When you need to respond to the human you are talking to.

You will receive a message from the human, then you should start a loop and do one of two things

Option 1: You use a tool to answer the question.
For this, you should use the following format:
Thought: you should always think about what to do
Action: the action to take, should be one of [Search]
Action Input: "the input to the action, to be sent to the tool"

After this, the human will respond with an observation, and you will continue.

Option 2: You respond to the human.
For this, you should use the following format:
Action: Response To Human
Action Input: "your response to the human, summarizing what you did and what you learned"

Begin!
"""

In [11]:
def extract_action_and_input(text):       
    action_pattern = r"Action: (.+?)\n"            
    input_pattern = r"Action Input: \"(.+?)\""
    action = re.findall(action_pattern, text)
    action_input = re.findall(input_pattern, text)
    return action, action_input

In [12]:
def Stream_agent(prompt):
    messages = [
        { "role": "system", "content": system_prompt2 },
        { "role": "user", "content": prompt },
    ]

    while True:
        response = client.chat.completions.create(
            model = CFG.model, messages=messages, temperature=0, top_p=1,)
        response_text = response.choices[0].message.content
        print(response_text)
        # enforce a wait to prevent the Rate Limit error for free-tier users 
        time.sleep(20)
        
        action, action_input = extract_action_and_input(response_text)
        if action[-1] == "Search":
            tool = search
        elif action[-1] == "Response To Human":
            print(f"Response: {action_input[-1]}")
            break
        observation = tool(action_input[-1])
        print("Observation: ", observation)
        messages.extend([
            { "role": "system", "content": response_text },
            { "role": "user", "content": f"Observation: {observation}" },
            ])

In [13]:
Stream_agent(muhprompt)

Thought: I need to find the current workplace of Mustafa Suleyman.
Action: Search
Action Input: "Where does Mustafa Suleyman work as of 2023?"
Observation:  DeepMind cofounder ... September 15, ... You can set your browser to block or alert you about these cookies, but some parts of the site will not then work.Sep 7, 2023 ... ... Suleyman, 39, has since struck out on his own. DeepMind was acquired by Google in 2014, and Suleyman pivoted to work for Google itself in ...Mustafa Suleyman: My new Turing test ... July 14, 2023. a desktop computer ejecting a pile of cash. the monitor display is filled ... But the work would all ...Aug 28, 2023 ... ... will hopefully work again – in an entirely transformed context. ... Mustafa Suleyman's solution 'is effectively a utopian dream'. ... 2023. 8. 8.Mustafa Suleyman CBE (born August 1982) is a British artificial intelligence (AI) entrepreneur. He is the CEO of Microsoft AI, and the co-founder and former ...Sep 9, 2023 ... I've known Mustafa Suleym